In [1]:
from lxml import etree
import pandas as pd
from io import StringIO

In [2]:
doc = etree.parse("fpds_feed_sample_1_org.xml")

In [3]:
xslt_tree = etree.parse("fpds_feed_v3.xsl")
transform = etree.XSLT(xslt_tree)

In [4]:
result = transform(doc)

In [5]:
csv_str =str(result)
csv_str

'agencyID\t9701\t9700\t9700\t9700\t9700\t9700\t9700\t9700\t9700\t9700\nagencyName\tDEPT OF DEFENSE\tDEPT OF DEFENSE\tDEPT OF DEFENSE\tDEPT OF DEFENSE\tDEPT OF DEFENSE\tDEPT OF DEFENSE\tDEPT OF DEFENSE\tDEPT OF DEFENSE\tDEPT OF DEFENSE\tDEPT OF DEFENSE\nPIID\t0001\t0001\t0001\t0001\t0001\t0001\t0001\t0001\t0001\t0001\nmodNumber\t0\t0\t0\t0\t0\t0\t0\t0\t0\t11\ntransactionNumber\t0\t0\t0\t0\t0\t0\t0\t0\t0\t0\nref-agencyID\t9700\t9700\t9700\t9700\t9700\t9700\t9700\t9700\t9700\t9700\nref-agencyName\tDEPT OF DEFENSE\tDEPT OF DEFENSE\tDEPT OF DEFENSE\tDEPT OF DEFENSE\tDEPT OF DEFENSE\tDEPT OF DEFENSE\tDEPT OF DEFENSE\tDEPT OF DEFENSE\tDEPT OF DEFENSE\tDEPT OF DEFENSE\nref-PIID\tN0017811D6668\tN0017812D7077\tN0017814D7171\tN0017814D7174\tN0017815D8340\tN0017816D8902\tSPE4A716D0258\tW15P7T09DW201\tW81K0417A0003\tW91RUS15D0015\nref-modNumber\t0\t0\t0\t0\t0\t0\t0\t0\t0\t0\nother-agencyID\t\t\t\t\t\t\t\t\t\t\nother-agencyName\t\t\t\t\t\t\t\t\t\t\nother-PIID\t\t\t\t\t\t\t\t\t\t\nother-modNumber\t\t

In [6]:
mystr = StringIO(csv_str)

df = pd.read_csv(mystr, sep="\t", header=None, index_col=0)
df.tail(10)

,1,2,3,4,5,6,7,8,9,10
0,,,,,,,,,,
fundingRequestingAgency-departmentID,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
fundingRequestingAgency-departmentName,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
fundingRequestingOfficeID,N65540,N65540,N65540,N65540,N65540,N65540,SPE4A7,W81DJP,W45NSV,W5K9FE
fundingRequestingOffice-name,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
foreignFunding,X,X,X,X,X,X,X,X,X,X
foreignFunding-description,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
purchaseReason,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
purchaseReason-description,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
feePaidForUseOfService,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00


In [7]:
df_new = df.transpose(copy=True)
df_new.iloc[:, -10:]

,fundingRequestingAgency-departmentID,fundingRequestingAgency-departmentName,fundingRequestingOfficeID,fundingRequestingOffice-name,foreignFunding,foreignFunding-description,purchaseReason,purchaseReason-description,feePaidForUseOfService,totalEstimatedOrderValue
1,NaN,NaN,N65540,NaN,X,NaN,NaN,NaN,0.00,NaN
2,NaN,NaN,N65540,NaN,X,NaN,NaN,NaN,0.00,NaN
3,NaN,NaN,N65540,NaN,X,NaN,NaN,NaN,0.00,NaN
4,NaN,NaN,N65540,NaN,X,NaN,NaN,NaN,0.00,NaN
5,NaN,NaN,N65540,NaN,X,NaN,NaN,NaN,0.00,NaN
6,NaN,NaN,N65540,NaN,X,NaN,NaN,NaN,0.00,NaN
7,NaN,NaN,SPE4A7,NaN,X,NaN,NaN,NaN,0.00,NaN
8,NaN,NaN,W81DJP,NaN,X,NaN,NaN,NaN,0.00,NaN
9,NaN,NaN,W45NSV,NaN,X,NaN,NaN,NaN,0.00,NaN
10,NaN,NaN,W5K9FE,NaN,X,NaN,NaN,NaN,0.00,NaN


In [8]:
df_new.shape

(10, 48)

In [ ]:
# output dataframe to a .csv file
df_new.to_csv("fpds_feed_result.csv", index=False)